# Lab 5: Google Speech Commands

## Imports

In [25]:
import copy
import wave
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

## Download, cache and extract Google Speech Commands

In [26]:
dataset_dir = Path('datasets')
if not (dataset_dir/'testing_list.txt').exists(): # Assume dataset already downloaded/extracted if testing list is present
    get_file(None, "http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz",
                    extract=True,
                    file_hash="6b74f3901214cb2c2934e98196829835",
                    cache_dir='.',
                    cache_subdir=dataset_dir)

## Load raw spoken digits data from Google Speech Commands

In [27]:
# Classes to handle, ordered by label
CLASSES = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

with (dataset_dir/'testing_list.txt').open() as f:
    testing_list = f.read().splitlines()

x_train = []
y_train = []
x_test = []
y_test = []

for recording in dataset_dir.glob(f'**/*.wav'):
    if not recording.parent.name in CLASSES: # Ignore unused classes
        continue
    label = CLASSES.index(recording.parent.name) # Assign class number
    
    with wave.open(str(recording)) as f: # Read wave file
        data = np.frombuffer(f.readframes(f.getnframes()), dtype=np.int16).copy() # As 16-bit signed integer
        
    data = data.astype(np.float32) # Convert to 32-bit floating-point
    data.resize((16000, 1)) # Resize to 1s (16kHz) with zero-padding, 1 channel

    if str(recording.relative_to(dataset_dir)) in testing_list: # Assign to test set if file in test list
        x_test.append(data)
        y_test.append(label)
    else:
        x_train.append(data)
        y_train.append(label)

x_train = np.array(x_train)
y_train = to_categorical(np.array(y_train))
x_test = np.array(x_test)
y_test = to_categorical(np.array(y_test))

## Prepare for inference with fixed-point Q7.9 samples by scaling input data accordingly

In [28]:
FIXED_POINT = 9
x_train /= 2**FIXED_POINT
x_test  /= 2**FIXED_POINT

## Export small dataset (250 random vectors)

In [29]:
perms = np.random.permutation(len(y_test))[0:250]
x_test_250 = x_test[perms]
y_test_250 = y_test[perms]
np.savetxt('x_test_gsc_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_gsc_250.csv', y_test_250, delimiter=',', fmt='%s')

## Build model M5

In [32]:
model = Sequential()
model.add(Input(shape=(16000,1)))
model.add(Conv1D(kernel_size=32, filters=4, strides=4))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(kernel_size=8, filters=8, strides=1))
model.add(MaxPool1D(pool_size=4))


model.add(Flatten())
model.add(Dense(10))
model.add(Activation('softmax')) 

# EXPLORE Learning Rate
# callbacks = EarlyStopping(monitor = "val_loss", patience  = 5)
opt = tf.keras.optimizers.Adam(learning_rate=10e-3)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 3993, 4)           132       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 998, 4)           0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 991, 8)            264       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 247, 8)           0         
 1D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 1976)              0         
                                                                 
 dense_3 (Dense)             (None, 10)               

## Train model

In [33]:
model.fit(x_train, y_train, epochs=20, batch_size=384, validation_data=(x_test, y_test))

Epoch 1/20


2023-04-07 17:19:34.098724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


91/91 [==============================] - ETA: 0s - loss: 2.4790 - categorical_accuracy: 0.3355

2023-04-07 17:19:39.770974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


91/91 [==============================] - 6s 60ms/step - loss: 2.4790 - categorical_accuracy: 0.3355 - val_loss: 1.6167 - val_categorical_accuracy: 0.4653
Epoch 2/20
91/91 [==============================] - 5s 52ms/step - loss: 1.5028 - categorical_accuracy: 0.5135 - val_loss: 1.5871 - val_categorical_accuracy: 0.4624
Epoch 3/20
91/91 [==============================] - 5s 52ms/step - loss: 1.4464 - categorical_accuracy: 0.5331 - val_loss: 1.6257 - val_categorical_accuracy: 0.4838
Epoch 4/20
91/91 [==============================] - 5s 52ms/step - loss: 1.4254 - categorical_accuracy: 0.5344 - val_loss: 1.7117 - val_categorical_accuracy: 0.4505
Epoch 5/20
91/91 [==============================] - 5s 52ms/step - loss: 1.3604 - categorical_accuracy: 0.5573 - val_loss: 1.6934 - val_categorical_accuracy: 0.4843
Epoch 6/20
91/91 [==============================] - 5s 53ms/step - loss: 1.3140 - categorical_accuracy: 0.5680 - val_loss: 1.7081 - val_categorical_accuracy: 0.4777
Epoch 7/20
91/91 [===

## Evaluate model on test dataset

In [34]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

129/129 - 1s - loss: 1.8494 - categorical_accuracy: 0.4862 - 917ms/epoch - 7ms/step
 35/129 [=======>......................] - ETA: 0s

2023-04-07 17:21:21.674010: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - 1s 5ms/step
tf.Tensor(
[[194   3  49  28  18  14   8  59  31  14]
 [ 12 195   6  12  26  46   1   6   3  92]
 [ 78  12 114 103  16   2  10  16  64   9]
 [ 45   0  50 178   4   5  15  12  86  10]
 [ 19  72  35   9 221  27   0  12   3   2]
 [ 13 104   1   4  14 156   2  27   5 119]
 [ 11   3  15   7   2   1 234  63  58   0]
 [ 32  19  30  15  18   8  22 240   7  15]
 [ 25   2  36  75   6   2  29  12 217   4]
 [ 10  42   9  15   8  35   0  28  13 248]], shape=(10, 10), dtype=int32)


## Evaluate model on small dataset

In [35]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

8/8 - 0s - loss: 1.8199 - categorical_accuracy: 0.4560 - 86ms/epoch - 11ms/step
8/8 [==============================] - 0s 7ms/step
tf.Tensor(
[[10  0  3  1  5  2  1  3  1  0]
 [ 1 14  0  0  1  4  0  0  0  3]
 [ 6  1  7  5  2  0  0  1  6  0]
 [ 3  0  5 13  0  0  0  1  6  0]
 [ 2  6  2  1 17  0  0  1  0  0]
 [ 1  5  0  0  1  8  0  2  0  7]
 [ 1  1  1  1  0  0 12  5  6  0]
 [ 1  2  2  0  1  2  5 12  0  2]
 [ 1  0  2  4  0  0  2  0 10  0]
 [ 1  0  1  1  0  3  0  1  1 11]], shape=(10, 10), dtype=int32)


## Save trained model

In [36]:
model.save('lab_gsc.h5')

## Remove SoftMax layer

In [37]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [41]:
%pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip#subdirectory=third_party/kerascnn2c_fixed

import kerascnn2c

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.3 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.
ERROR: You must give at least one requirement to uninstall (see "pip help uninstall")
Note: you may need to restart the kernel to use updated packages.


## Generate C code for the trained model with 16-bit fixed-point representation

In [39]:
res = kerascnn2c.Converter(output_path=Path('gsc_output_fixed'),
                           fixed_point=FIXED_POINT, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('gsc_model_fixed.h', 'w') as f:
    f.write(res)

INFO:tensorflow:Assets written to: ram://9b2d956c-1708-496d-9c47-1c8cb4fc5b2c/assets


INFO:tensorflow:Assets written to: ram://9b2d956c-1708-496d-9c47-1c8cb4fc5b2c/assets


———————————————————————————————————————————————————————————————————————————————————————————————————————
Inputs                           | Layer                            | Outputs                         
———————————————————————————————————————————————————————————————————————————————————————————————————————
                                 | input_6                          | conv1d_4                        
-------------------------------------------------------------------------------------------------------
input_6                          | conv1d_4                         | max_pooling1d_2                 
-------------------------------------------------------------------------------------------------------
conv1d_4                         | max_pooling1d_2                  | conv1d_5                        
-------------------------------------------------------------------------------------------------------
max_pooling1d_2                  | conv1d_5                         

## Compile the 16-bit fixed-point C code for x86 and evaluate on small dataset

In [40]:
!g++ -Wall -Wextra -pedantic -Ofast -o gsc_fixed -Igsc_output_fixed/ gsc_output_fixed/model.c main.cpp 
!./gsc_fixed x_test_gsc_250.csv y_test_gsc_250.csv

clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]
clang: error: no such file or directory: 'main.cpp'
zsh:1: no such file or directory: ./gsc_fixed
